In [2]:
from pathlib import Path
from collections import Counter
import json
import pandas as pd
import csv
import os

IMAGE_DIR = os.path.join('.', 'data\\images\\')

In [2]:
files=pd.read_csv('checkfile.csv')

In [5]:
files['path']= IMAGE_DIR + files['image_id'] + ".JPG"

In [7]:
files['exist']=0

In [10]:
from os.path import exists

files['exist'] = files['path'].map(lambda x: exists(x))




In [13]:
files.to_csv('exisingfiles.csv')

In [8]:
files

,image_id,nebu2,pix1,pix2,age,localization,bm,dx,nebu1,tf,del,path,exist
0,ISIC_0000000,dermoscopic,1022,767,55.0,anterior torso,benign,nevus,NaN,True,NaN,.\data\images\ISIC_0000000.JPG,0
1,ISIC_0000001,dermoscopic,1022,767,30.0,anterior torso,benign,nevus,NaN,True,NaN,.\data\images\ISIC_0000001.JPG,0
2,ISIC_0000002,dermoscopic,1022,767,60.0,upper extremity,malignant,melanoma,histopathology,True,NaN,.\data\images\ISIC_0000002.JPG,0
3,ISIC_0000003,dermoscopic,1022,767,30.0,upper extremity,benign,nevus,NaN,True,NaN,.\data\images\ISIC_0000003.JPG,0
4,ISIC_0000004,dermoscopic,1022,767,80.0,posterior torso,malignant,melanoma,histopathology,True,NaN,.\data\images\ISIC_0000004.JPG,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60497,ISIC_9995691,dermoscopic,1872,1053,50.0,upper extremity,benign,nevus,serial imaging showing no change,True,NaN,.\data\images\ISIC_9995691.JPG,0
60498,ISIC_9997614,NaN,3264,2448,50.0,upper extremity,benign,nevus,histopathology,True,NaN,.\data\images\ISIC_9997614.JPG,0
60499,ISIC_9998682,NaN,4032,3024,60.0,head/neck,malignant,melanoma,histopathology,True,NaN,.\data\images\ISIC_9998682.JPG,0
60500,ISIC_9998937,dermoscopic,1872,1053,40.0,head/neck,benign,nevus,serial imaging showing no change,True,NaN,.\data\images\ISIC_9998937.JPG,0


In [4]:


dir = Path('Descriptions')
#out_file = Path('output.txt')

#data=pd.read_csv('imagemain.csv')


import json




for file in dir.glob('*'):
    with file.open('r', encoding='utf-8') as f:
        _f = json.load(f)
        
        _h = []
        
        # _h.append(_f["id"])
        try:
            _h.append(_f["name"])
        except:
            _h.append('')

        try:
            _h.append(_f["meta"]["acquisition"]["image_type"])
        except:
            _h.append('')

        try:
            _h.append(_f["meta"]["acquisition"]["pixelsX"])
        except:
            _h.append('')

        try:
            _h.append(_f["meta"]["acquisition"]["pixelsY"])
        except:
            _h.append('')

        try:
            _h.append(_f["meta"]["clinical"]["age_approx"])
        except:
            _h.append('')
            
        try:
            _h.append(_f["meta"]["clinical"]["anatom_site_general"])
        except:
            _h.append('')

        try:
            _h.append(_f["meta"]["clinical"]["benign_malignant"])
        except:
            _h.append('')
        
        try:
            _h.append(_f["meta"]["clinical"]["diagnosis"])
        except:
            _h.append('')

        try:
            _h.append(_f["meta"]["clinical"]["diagnosis_confirm_type"])
        except:
            _h.append('')
            
        try:
            _h.append(_f["meta"]["clinical"]["melanocytic"])
        except:
            _h.append('')
        
        with open('_prop4.csv', 'a', newline='') as f:
                    write = csv.writer(f)
                    write.writerow(_h)
        
        
     

        



In [6]:
data

,lesion_id,image_id,dx,dx_type,age,sex,localization


In [14]:
from PIL import Image
import os, sys

path = "./data/images/"
dirs = os.listdir( path )

files=pd.read_csv('data/HAM10000_metadata.csv')



def resize():
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((640,480))
            imResize.save(f + ' r.jpg', 'JPEG', quality=90)



In [19]:
files

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face


In [45]:
files['dx'].value_counts(dropna=False)

nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: dx, dtype: int64

In [27]:
a=dict(files['dx'].value_counts())

In [30]:
files

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face


In [32]:
IMAGE_DIR = os.path.join('.', 'data\\HAM10000_images\\')
files['path']= IMAGE_DIR + files['image_id'] + ".JPG"

In [37]:
files['destination'] = IMAGE_DIR+ files['dx'] + "\\" + files['image_id'] + ".JPG"

In [38]:
files

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,destination
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,.\data\HAM10000_images\ISIC_0027419.JPG,.\data\HAM10000_images\bkl\ISIC_0027419.JPG
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,.\data\HAM10000_images\ISIC_0025030.JPG,.\data\HAM10000_images\bkl\ISIC_0025030.JPG
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,.\data\HAM10000_images\ISIC_0026769.JPG,.\data\HAM10000_images\bkl\ISIC_0026769.JPG
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,.\data\HAM10000_images\ISIC_0025661.JPG,.\data\HAM10000_images\bkl\ISIC_0025661.JPG
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,.\data\HAM10000_images\ISIC_0031633.JPG,.\data\HAM10000_images\bkl\ISIC_0031633.JPG
...,...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,.\data\HAM10000_images\ISIC_0033084.JPG,.\data\HAM10000_images\akiec\ISIC_0033084.JPG
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,.\data\HAM10000_images\ISIC_0033550.JPG,.\data\HAM10000_images\akiec\ISIC_0033550.JPG
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,.\data\HAM10000_images\ISIC_0033536.JPG,.\data\HAM10000_images\akiec\ISIC_0033536.JPG
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,.\data\HAM10000_images\ISIC_0032854.JPG,.\data\HAM10000_images\akiec\ISIC_0032854.JPG


In [46]:
import shutil
for a in files.values:
    #print(a[7])
    shutil.move(a[7], a[8])

In [6]:
from os.path import exists

#file_exists = exists(path_to_file)


In [7]:
files['exists']=files['path'].map(lambda x: exists(x))

In [47]:
files.to_csv("_prop2.csv")

In [ ]:
import splitfolders

In [50]:
files

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,destination
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,.\data\HAM10000_images\ISIC_0027419.JPG,.\data\HAM10000_images\bkl\ISIC_0027419.JPG
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,.\data\HAM10000_images\ISIC_0025030.JPG,.\data\HAM10000_images\bkl\ISIC_0025030.JPG
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,.\data\HAM10000_images\ISIC_0026769.JPG,.\data\HAM10000_images\bkl\ISIC_0026769.JPG
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,.\data\HAM10000_images\ISIC_0025661.JPG,.\data\HAM10000_images\bkl\ISIC_0025661.JPG
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,.\data\HAM10000_images\ISIC_0031633.JPG,.\data\HAM10000_images\bkl\ISIC_0031633.JPG
...,...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,.\data\HAM10000_images\ISIC_0033084.JPG,.\data\HAM10000_images\akiec\ISIC_0033084.JPG
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,.\data\HAM10000_images\ISIC_0033550.JPG,.\data\HAM10000_images\akiec\ISIC_0033550.JPG
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,.\data\HAM10000_images\ISIC_0033536.JPG,.\data\HAM10000_images\akiec\ISIC_0033536.JPG
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,.\data\HAM10000_images\ISIC_0032854.JPG,.\data\HAM10000_images\akiec\ISIC_0032854.JPG


In [49]:

from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array



In [61]:


datagen= ImageDataGenerator(rescale = 1./255,
                                  rotation_range = 10,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  shear_range = 0.2,
                                  horizontal_flip = True,
                                  vertical_flip = True,
                                  fill_mode = 'nearest')


In [68]:
IMAGE_DIR = os.path.join('.', 'data\\HAM10000_images\\vasc\\')
count = 0
for batch in datagen.flow_from_directory(directory=IMAGE_DIR,  class_mode=None, batch_size=40,save_to_dir=IMAGE_DIR, save_prefix='g', save_format='jpg'):
    count += 1
    if count > 164:
        break


Found 142 images belonging to 2 classes.


In [69]:
import splitfolders
IMAGE_DIR = os.path.join('.', 'data\\HAM10000_images\\')
splitfolders.ratio(IMAGE_DIR,'IMAGES',seed=42, ratio=(0.7,0.2,0.1))